# Automatic generation of Notebook using PyCropML
    This notebook implements a crop model.

### Model Rangeofsoiltemperaturesdaycent

In [ ]:
MODULE list_sub
    IMPLICIT NONE
    TYPE container
        CLASS(*), ALLOCATABLE :: item
        CLASS(*), ALLOCATABLE :: items(:)
    END TYPE

    interface Add
        module procedure AddToListFloat
        module procedure AddToListInt
        module procedure AddToListChar
        module procedure AddToListIntArray
    end interface
CONTAINS

    FUNCTION indice(vectorElem, elem)
        CHARACTER(LEN=*), DIMENSION(:):: vectorElem
        INTEGER::iterator, indice
        CHARACTER(LEN=*):: elem
        DO iterator= 1, SIZE(vectorElem)
            IF(vectorElem(iterator)==elem) THEN
                indice = iterator
            END IF
        END DO
        RETURN
    END FUNCTION indice


    SUBROUTINE AddToListFloat(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        REAL, INTENT(IN) :: element
        REAL, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        REAL, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListFloat

    SUBROUTINE AddToListInt(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        INTEGER, INTENT(IN) :: element
        INTEGER, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        INTEGER, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListInt

    SUBROUTINE AddToListChar(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize, l
        CHARACTER(LEN=*), INTENT(IN) :: element
        CHARACTER(LEN=*), DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        CHARACTER(LEN=65), DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            l=1
            ALLOCATE(list(l))
            list(l) = element
        END IF
    END SUBROUTINE AddToListChar


    SUBROUTINE AddToListIntArray(a, e)
        INTEGER, ALLOCATABLE, INTENT(INOUT) :: a(:)
        INTEGER, ALLOCATABLE, INTENT(IN) :: e(:)
        INTEGER, ALLOCATABLE:: tmp(:)

        IF (.NOT.ALLOCATED(a)) THEN
            ALLOCATE(a(1))
            ALLOCATE(a(SIZE(e)), source = e)
        ELSE
            CALL MOVE_ALLOC(a,tmp)
            ALLOCATE(a(SIZE(tmp)+SIZE(e)), source = e)
            a(1:SIZE(tmp)) = tmp
            a(SIZE(tmp)+1:SIZE(a)) = e
        END IF
    END SUBROUTINE AddToListIntArray

END MODULE list_sub
MODULE Rangeofsoiltemperaturesdaycentmod
    USE list_sub
    IMPLICIT NONE
CONTAINS

    SUBROUTINE init_rangeofsoiltemperaturesdaycent(LayerThickness, &
        SoilTemperatureRangeByLayers, &
        SoilTemperatureMinimum, &
        SoilTemperatureMaximum)
        IMPLICIT NONE
        INTEGER:: i_cyml_r
        REAL , DIMENSION(: ), INTENT(IN) :: LayerThickness
        REAL , DIMENSION(: ), INTENT(OUT) :: SoilTemperatureRangeByLayers
        REAL , DIMENSION(: ), INTENT(OUT) :: SoilTemperatureMinimum
        REAL , DIMENSION(: ), INTENT(OUT) :: SoilTemperatureMaximum
        SoilTemperatureRangeByLayers = 0.0
        SoilTemperatureMaximum = 0.0
        SoilTemperatureMinimum = 0.0
    END SUBROUTINE init_rangeofsoiltemperaturesdaycent

    SUBROUTINE model_rangeofsoiltemperaturesdaycent(LayerThickness, &
        SurfaceTemperatureMinimum, &
        ThermalDiffusivity, &
        SoilTemperatureByLayers, &
        SurfaceTemperatureMaximum, &
        SoilTemperatureRangeByLayers, &
        SoilTemperatureMinimum, &
        SoilTemperatureMaximum)
        IMPLICIT NONE
        INTEGER:: i_cyml_r
        REAL , DIMENSION(: ), INTENT(IN) :: LayerThickness
        REAL, INTENT(IN) :: SurfaceTemperatureMinimum
        REAL , DIMENSION(: ), INTENT(IN) :: ThermalDiffusivity
        REAL , DIMENSION(: ), INTENT(IN) :: SoilTemperatureByLayers
        REAL, INTENT(IN) :: SurfaceTemperatureMaximum
        REAL , DIMENSION(: ), INTENT(INOUT) :: SoilTemperatureRangeByLayers
        REAL , DIMENSION(: ), INTENT(INOUT) :: SoilTemperatureMinimum
        REAL , DIMENSION(: ), INTENT(INOUT) :: SoilTemperatureMaximum
        INTEGER:: i
        REAL:: _DepthBottom
        REAL:: _DepthCenterLayer
        REAL:: SurfaceDiurnalRange
        !- Name: RangeOfSoilTemperaturesDAYCENT -Version: 001, -Time step: 1
        !- Description:
    !            * Title: RangeOfSoilTemperaturesDAYCENT model
    !            * Authors: simone.bregaglio
    !            * Reference: http://bioma.jrc.ec.europa.eu/ontology/JRC_MARS_biophysical_domain.owl
    !            * Institution: University Of Milan
    !            * ExtendedDescription: Strategy for the calculation of soil thermal conductivity.Reference: DAYCENT model written in C code
    !            * ShortDescription: Strategy for the calculation of soil thermal conductivity
        !- inputs:
    !            * name: LayerThickness
    !                          ** description : Soil layer thickness
    !                          ** inputtype : parameter
    !                          ** parametercategory : constant
    !                          ** datatype : DOUBLEARRAY
    !                          ** len : 
    !                          ** max : 3
    !                          ** min : 0.005
    !                          ** default : 0.05
    !                          ** unit : m
    !            * name: SurfaceTemperatureMinimum
    !                          ** description : Minimum surface soil temperature
    !                          ** inputtype : variable
    !                          ** variablecategory : auxiliary
    !                          ** datatype : DOUBLE
    !                          ** max : 60
    !                          ** min : -60
    !                          ** default : 10
    !                          ** unit : degC
    !            * name: ThermalDiffusivity
    !                          ** description : Thermal diffusivity of soil layer
    !                          ** inputtype : variable
    !                          ** variablecategory : auxiliary
    !                          ** datatype : DOUBLEARRAY
    !                          ** len : 
    !                          ** max : 1
    !                          ** min : 0
    !                          ** default : 0.0025
    !                          ** unit : mm s-1
    !            * name: SoilTemperatureByLayers
    !                          ** description : Soil temperature of each layer
    !                          ** inputtype : variable
    !                          ** variablecategory : auxiliary
    !                          ** datatype : DOUBLEARRAY
    !                          ** len : 
    !                          ** max : 60
    !                          ** min : -60
    !                          ** default : 15
    !                          ** unit : degC
    !            * name: SurfaceTemperatureMaximum
    !                          ** description : Maximum surface soil temperature
    !                          ** inputtype : variable
    !                          ** variablecategory : auxiliary
    !                          ** datatype : DOUBLE
    !                          ** max : 60
    !                          ** min : -60
    !                          ** default : 25
    !                          ** unit : degC
    !            * name: SoilTemperatureRangeByLayers
    !                          ** description : Soil temperature range by layers
    !                          ** inputtype : variable
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLEARRAY
    !                          ** len : 
    !                          ** max : 50
    !                          ** min : 0
    !                          ** default : 
    !                          ** unit : degC
    !            * name: SoilTemperatureMinimum
    !                          ** description : Minimum soil temperature by layers
    !                          ** inputtype : variable
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLEARRAY
    !                          ** len : 
    !                          ** max : 60
    !                          ** min : -60
    !                          ** default : 
    !                          ** unit : degC
    !            * name: SoilTemperatureMaximum
    !                          ** description : Maximum soil temperature by layers
    !                          ** inputtype : variable
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLEARRAY
    !                          ** len : 
    !                          ** max : 60
    !                          ** min : -60
    !                          ** default : 
    !                          ** unit : degC
        !- outputs:
    !            * name: SoilTemperatureRangeByLayers
    !                          ** description : Soil temperature range by layers
    !                          ** datatype : DOUBLEARRAY
    !                          ** variablecategory : state
    !                          ** len : 
    !                          ** max : 50
    !                          ** min : 0
    !                          ** unit : degC
    !            * name: SoilTemperatureMinimum
    !                          ** description : Minimum soil temperature by layers
    !                          ** datatype : DOUBLEARRAY
    !                          ** variablecategory : state
    !                          ** len : 
    !                          ** max : 60
    !                          ** min : -60
    !                          ** unit : degC
    !            * name: SoilTemperatureMaximum
    !                          ** description : Maximum soil temperature by layers
    !                          ** datatype : DOUBLEARRAY
    !                          ** variablecategory : state
    !                          ** len : 
    !                          ** max : 60
    !                          ** min : -60
    !                          ** unit : degC
        _DepthBottom = REAL(0)
        _DepthCenterLayer = REAL(0)
        SurfaceDiurnalRange = SurfaceTemperatureMaximum -  &
                SurfaceTemperatureMinimum
        DO i = 0 , SIZE(SoilTemperatureByLayers)-1, 1
            IF(i .EQ. 0) THEN
                _DepthCenterLayer = LayerThickness(1) * 100 / 2
                SoilTemperatureRangeByLayers(1) = SurfaceDiurnalRange *  &
                        EXP((-_DepthCenterLayer) *  ((0.00005 / ThermalDiffusivity(1)) **  &
                        0.5))
                SoilTemperatureMaximum(1) = SoilTemperatureByLayers(1) +  &
                        (SoilTemperatureRangeByLayers(1) / 2)
                SoilTemperatureMinimum(1) = SoilTemperatureByLayers(1) -  &
                        (SoilTemperatureRangeByLayers(1) / 2)
            ELSE
                _DepthBottom = _DepthBottom + (LayerThickness((i - 1)+1) * 100)
                _DepthCenterLayer = _DepthBottom + (LayerThickness(i+1) * 100 / 2)
                SoilTemperatureRangeByLayers(i+1) = SurfaceDiurnalRange *  &
                        EXP((-_DepthCenterLayer) *  ((0.00005 / ThermalDiffusivity(i+1)) **  &
                        0.5))
                SoilTemperatureMaximum(i+1) = SoilTemperatureByLayers(i+1) +  &
                        (SoilTemperatureRangeByLayers(i+1) / 2)
                SoilTemperatureMinimum(i+1) = SoilTemperatureByLayers(i+1) -  &
                        (SoilTemperatureRangeByLayers(i+1) / 2)
            END IF
        END DO
    END SUBROUTINE model_rangeofsoiltemperaturesdaycent

END MODULE
PROGRAM test
    USE Rangeofsoiltemperaturesdaycentmod

END PROGRAM
